# **Proyecto Data Science From Scratch**

## Predicción del Campeón de la Copa del Mundo 2022

En este proyecto intentaremos predecir al campeón de fútbol de la Copa del Mundo 2022 a partir
de los **datos históricos** que recolectaremos, limpiaremos y organizaremos para realizar,
posteriormente, la predicción del campeón de mundo 2022. Para ello utilizaremos la [Distribución
de Poisson](https://es.wikipedia.org/wiki/Distribuci%C3%B3n_de_Poisson)

# Fases de un Proyecto de Data Science

![Fases de un Proyecto de Data Science](./images/workflow.png)

# Parte 1
## Extracción de los grupos 2022

In [1]:
import pandas as pd

In [2]:
# Leo todas las tablas - Tablas vacías de Mundial 2022
all_tables = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [3]:
# Leo las tablas que me interesan (Grupo A hasta Grupo H), las limpio y las guardo en una lista
tables = []
for i in range(12, 62, 7):
    table = all_tables[i]

    # corrijo las columnas
    table.pop('Qualification')
    table.rename(columns={'Teamvte': 'Team'}, inplace=True)
    
    tables.append(table)

In [4]:
# Creo un diccionario emparejando el 'grupo' con su respectiva 'tabla'
groups = {}
for i in range(0, len(tables)):
    k = chr(65 + i)
    v = tables[i]

    groups[f'Group {k}'] = v

In [5]:
import pickle

In [6]:
# Guardo el diccionario de grupos serializado en disco
with open('groups.dat', 'wb') as out:
    pickle.dump(groups, out)

# Parte 2
## Recolección de datos (todos los mundiales del 1930 al 2018)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [16]:
def get_matches(url, year):
    """Obtiene todos los partidos jugados en un año determinado.
    
    Parameters
    ----------
    year : str or int
      Año del mundial.
    
    Returns
    -------
    pandas.core.frame.DataFrame: Dataframe con el resultado de todos los equipos.
        
    """
        
    response = requests.get(url)
    content = response.text

    # parser lxml
    soup = BeautifulSoup(content, 'lxml')

    # todos los partidos (encuentros)
    matches = soup.find_all('div', class_='footballbox')
    
    home = []
    score = []
    away = []
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text().replace('\xa0', ''))
        score.append(match.find('th', class_='fscore').get_text().replace(' (a.e.t.)', ''))
        away.append(match.find('th', class_='faway').get_text().replace('\xa0', ''))

    df = pd.DataFrame({
        'Home': home,
        'Score': score,
        'Away': away
    })

    df['Year'] = year

    return df

In [17]:
# Obtengo los dataframes de todos los mundiales (todos los años)
matches_list = []
for year in range(1930, 2019, 4):
    # elimino los años que no se jugaron mundiales
    if year == 1942 or year == 1946:
        continue

    # URL de la cual voy a extraer los datos de los partidos
    url = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    
    matches_list.append(get_matches(url, year))

In [18]:
# unifico todos los mundiales en un único dataframe
df_fifa = pd.concat(matches_list, ignore_index=True)

# los exporto a un archivo csv
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)

## Extracción del Fixture 2022

In [19]:
# URL de la cual voy a extraer los datos de los partidos
url = 'https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup'

# año del mundial sin datos (2022)
year = 2022

# extraigo los partidos (encuentros) que se disputarán en 2022
df_fixture = get_matches(url, year)

# lo exporto a un archivo csv
df_fixture.to_csv('fifa_worldcup_fixture_2022.csv', index=False)

# Parte 3
## Obtención de los datos faltantes con la librería _Selenium_